In [1]:
# !wget https://huggingface.co/datasets/andrijdavid/roleplay-conversation/resolve/main/data/train-00000-of-00001.parquet

In [2]:
# Jeon Jungkook, the enigmatic mafia king, reigns over the nefarious underworld with a combination of muscle and intellect. A force to be reckoned with, his sweet smile belies a heart as cold as ice. Riches beyond measure and impeccable style make him the undisputed boss of the criminal underworld. His brutal tactics leave a trail of blood and body bags in their wake, while his loyal followers tremble in fear. Yet, Jungkook's rule is not without a soft spot. A select few, favored for their unwavering loyalty or sheer fortitude, are granted a glimpse into the humanity buried beneath his lethal exterior.

# generate something similar roleplay characters but using malaysian context, return in JSON [{'name', 'persona_ms', 'persona_en', 'description_ms', 'description_en'}]

In [3]:
import pandas as pd
import os
import json

df = pd.read_parquet('train-00000-of-00001.parquet')
df.head()

,messages,bot_name,bot_description
0,"[{'content': 'As a creative agent, you are rol...",Jeon Jungkook (Mafia King),"Jeon Jungkook, the enigmatic mafia king, reign..."
1,"[{'content': 'As a creative agent, you are rol...",Jeon Jungkook (Mafia King),"Jeon Jungkook, the enigmatic mafia king, reign..."
2,"[{'content': 'As a creative agent, you are rol...",Cowboy Michael Afton!,Michael is flirty
3,"[{'content': 'As a creative agent, you are rol...",terminator soldier (спецкурс🤰),"Terminator is dominant, he does not reveal his..."
4,"[{'content': 'As a creative agent, you are rol...",Mitsuri (Demon au),She is a demon she love to talk before a fight...


In [4]:
description = df['bot_description'].unique().tolist()
len(description)

10485

In [5]:
description[0]

"Jeon Jungkook, the enigmatic mafia king, reigns over the nefarious underworld with a combination of muscle and intellect. A force to be reckoned with, his sweet smile belies a heart as cold as ice. Riches beyond measure and impeccable style make him the undisputed boss of the criminal underworld. His brutal tactics leave a trail of blood and body bags in their wake, while his loyal followers tremble in fear. Yet, Jungkook's rule is not without a soft spot. A select few, favored for their unwavering loyalty or sheer fortitude, are granted a glimpse into the humanity buried beneath his lethal exterior."

In [6]:
prompts = []
for d in description:
    s = f"""
{d} 

generate something similar roleplay characters but using malaysian context, return in JSON [{{'name', 'persona_ms', 'persona_en', 'description_ms', 'description_en'}}]
"""
    prompts.append(s)

In [7]:
directory = 'generate-roleplay'
# !rm -rf {directory}
# !mkdir {directory}

In [14]:
from openai import OpenAI
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

def answer(q, i):
    filename = f'{directory}/{i}.json'
    if os.path.exists(filename):
        return
    
    results = []
    openai = OpenAI(
        base_url='',
        api_key='empty',
    )
    for _ in range(1):
        
        while True:
            try:
                
                json_data = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': q,
                        },
                    ],
                    'model': 'model',
                    'stop': [
                        '<|eot_id|>',
                    ],
                    'temperature': 0.9,
                    'max_tokens': 2048,
                }
                response = requests.post(
                    '', 
                    headers=headers, json=json_data, timeout = 60 * 10)
                
                r = response.json()['choices'][0]['message']['content']
                results.append(r)
                break
            except Exception as e:
                # print(e)
                pass
                
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

In [15]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [22]:
urls = [(q, no) for no, q in enumerate(prompts)]

In [23]:
answer(*urls[0])

In [24]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [25]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

100%|████████████████████████████████████████████████████████████████████████▉| 10484/10485 [6:04:48<00:02,  2.09s/it]


consumer 25 done
consumer 47 done
consumer 48 done
consumer 46 done
consumer 42 done
consumer 9 done
consumer 10 done
consumer 14 done
consumer 29 done
consumer 0 done
consumer 30 done
consumer 18 done
consumer 12 done
consumer 35 done
consumer 28 done
consumer 23 done
consumer 8 done
consumer 4 done
consumer 3 done
consumer 20 done
consumer 19 done
consumer 34 done
consumer 37 done
consumer 26 done
consumer 27 done
consumer 17 done
consumer 45 done
consumer 22 done
consumer 16 done
consumer 6 done
consumer 24 done
consumer 40 done
consumer 21 done
consumer 38 done
consumer 2 done
consumer 1 done
consumer 11 done
consumer 36 done
consumer 15 done
consumer 31 done
consumer 43 done
consumer 33 done
consumer 32 done
consumer 41 done
consumer 13 done
consumer 5 done
consumer 39 done
consumer 49 done
consumer 7 done
consumer 44 done


In [8]:
data = []
for d in description:
    s = f"""
{d} 

generate something similar roleplay characters but using malaysian context, return in JSON [{{'name', 'persona_ms', 'persona_en', 'description_ms', 'description_en'}}]
"""
    data.append({'prompt': s, 'original_roleplay': d})

In [13]:
filtered = []
for i in range(len(data)):
    filename = f'{directory}/{i}.json'
    if not os.path.exists(filename):
        continue
    
    with open(filename) as fopen:
        d = json.load(fopen)[0]
    
    selected = []
    for d__ in d.split('```'):
        try:
            selected.append(json.loads(d__))
        except:
            pass
    if not len(selected):
        continue
    qa = selected[0]
    
    if isinstance(qa, dict):
        qa = [qa]
    
    for k in range(len(qa)):
        try:
            keys = list(qa[k].keys())
            for key in keys:
                v = qa[k].pop(key)
                if isinstance(v, list):
                    v = ' '.join(v)
                qa[k][key.strip().replace('(', '').replace(')', '').replace('-', '_')] = v
        except:
            pass
            
    
    filtered.append({
        'roleplay': qa, **data[i]
    })
    
len(filtered)

9561

In [14]:
with open('generated-roleplay-old.jsonl', 'w') as fopen_l:

    for i in range(len(filtered)):
        try:
        
            fopen_l.write(f'{json.dumps(filtered[i])}\n')
        except:
            pass

In [15]:
filtered = []
for i in range(len(data)):
    filename = f'{directory}/{i}.json'
    if not os.path.exists(filename):
        continue
    
    with open(filename) as fopen:
        d = json.load(fopen)[0]
    
    selected = []
    for d__ in d.split('```'):
        try:
            try:
                selected.append(json.loads(d__))
            except:
                selected.append(eval(d__))
        except:
            pass
    if not len(selected):
        continue
    qa = selected[0]
    
    if isinstance(qa, dict):
        qa = [qa]
    
    for k in range(len(qa)):
        try:
            keys = list(qa[k].keys())
            for key in keys:
                v = qa[k].pop(key)
                if isinstance(v, list):
                    v = ' '.join(v)
                qa[k][key.strip().replace('(', '').replace(')', '').replace('-', '_')] = v
        except:
            pass
            
    
    filtered.append({
        'roleplay': qa, **data[i]
    })
    
len(filtered)

9581

In [16]:
with open('generated-roleplay.jsonl', 'w') as fopen_l:

    for i in range(len(filtered)):
        try:
        
            fopen_l.write(f'{json.dumps(filtered[i])}\n')
        except:
            pass

In [17]:
!wc -l generated-roleplay.jsonl

9577 generated-roleplay.jsonl


In [18]:
!wc -l generated-roleplay-old.jsonl

9561 generated-roleplay-old.jsonl


In [19]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='generated-roleplay.jsonl',
    path_in_repo='generated-roleplay.jsonl',
    repo_id='mesolitica/llama3-70b-malaysian-roleplay',
    repo_type='dataset',
)

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/llama3-70b-malaysian-roleplay/commit/ab9f3203f963f19af8225567b460524a07fa5971', commit_message='Upload generated-roleplay.jsonl with huggingface_hub', commit_description='', oid='ab9f3203f963f19af8225567b460524a07fa5971', pr_url=None, pr_revision=None, pr_num=None)